In [293]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2
import os, sys, fnmatch
import cv2, time, math, pickle
import numpy as np
import argparse, pprint

from matplotlib import pyplot as plt
import matplotlib
from matplotlib.ticker import NullFormatter
from matplotlib.transforms import Bbox
import matplotlib.gridspec as gridspec

import ipywidgets as widgets
from IPython.display import display, HTML, Math
from ipywidgets import interact, interactive, IntSlider, Layout, interact_manual,interact, HBox, Layout,VBox

from scripts.hyutils.sys_utils import *
from scripts.hyutils.img_utils import *
from scripts.hyutils.debug_utils import *
from scripts.hyutils.plotting_utils import *
from scripts.hyutils.uv_mapping_utils import *

from scripts.VBOATS import VBOATS

vboat = VBOATS()
vboat.testMaxGndStep = 16

pp = pprint.PrettyPrinter(indent=4)
homeDir = os.path.abspath('')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [326]:
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/test_cases")
# found_imgs, _ = grab_dir_images("/home/hunter/devel/vision_playground/test/frames")
# found_imgs, _ = grab_dir_images("/home/hunter/data/images/obstacles")
# found_imgs, _ = grab_dir_images("/home/hunter/data/rosbags/extracted/depth")
found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_1")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hunter/data/vboats/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/hyoung/Documents/rosbags/extracted/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/devel/vision_playground/test/iai/obstacles")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/tmp/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/distance_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/angle_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/d415/tests/misc_test/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/test_keyParams")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_1")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/test_pool_2")

# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/misc/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/huh/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_0/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_1/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_2/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_3/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_4/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_5/depth")
# found_imgs, _ = grab_dir_images("/home/hyoung/data/vboats/data/tests/ng/test_6/depth")

#################################################################################################
#################################################################################################

# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/test_cases")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion")
# found_imgs, _ = grab_dir_images("/home/offset/Pictures/vboats/aion/huh")

sFImgs = sorted(found_imgs)

In [327]:
style = {'description_width': 'initial'}
layStretch = Layout(flex='1 1 auto')
layCenter = Layout(align_self='center',display = 'flex',align_items='stretch',flex='1 1 auto')
layWidth = Layout(width="auto",align_self='center')

imgPathSelector = widgets.Select(options=sFImgs,value=str(sFImgs[0]),description='File:', layout=Layout(width='100%'))

lblSz1 = widgets.Label(value="Deadzone Sizes (U,V):", layout=layCenter)
sz1Selector = widgets.IntSlider(value=0,min=0,max=20,step=1,description=' ', layout = layCenter)
sz2Selector = widgets.IntSlider(value=3,min=0,max=20,step=1,description=' ', layout = layCenter)

lblSz2 = widgets.Label(value="Strip[0] Deadzone Sizes (U,V):", layout=layCenter)
sz3Selector = widgets.IntSlider(value=4,min=0,max=20,step=1,description=' ', layout = layStretch)
sz4Selector = widgets.IntSlider(value=5,min=0,max=20,step=1,description=' ', layout = layStretch)

lblU = widgets.Label(value="UMap Strip Thresholds (0,1):", layout=layCenter)
u1threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout= layStretch)
# u1threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ',layout= layStretch)
u2threshSelector = widgets.FloatSlider(value=0.1,min=0,max=1,step=0.01,description=' ',layout = layStretch)

lblV = widgets.Label(value="VMap Strip Thresholds (0,1):", layout=layCenter)
v1threshSelector = widgets.IntSlider(value=5,min=0,max=255,step=1,description=' ',layout = layStretch)
v2threshSelector = widgets.IntSlider(value=70,min=0,max=255,step=1,description=' ',layout = layStretch)

lblD = widgets.Label(value="Deadzone Thresholds (U1, U2, V):", layout=layCenter)
d1threshSelector = widgets.FloatSlider(value=0.25,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d2threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)
d3threshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)

lblK = widgets.Label(value="Kernel Sizes:", layout=layCenter)
k11Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k11', layout = layStretch)
k12Selector = widgets.IntSlider(value=2,min=0,max=100,step=1,description='k12', layout = layStretch)
k21Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k21', layout = layStretch)
k22Selector = widgets.IntSlider(value=5,min=0,max=100,step=1,description='k22', layout = layStretch)
k31Selector = widgets.IntSlider(value=50,min=0,max=100,step=1,description='k31', layout = layStretch)
k32Selector = widgets.IntSlider(value=1,min=0,max=100,step=1,description='k32', layout = layStretch)

win1Selector = widgets.IntSlider(value=10,min=0,max=100,step=1,description='win1', layout = layStretch)
win1ThreshSelector = widgets.FloatSlider(value=0.15,min=0,max=1,step=0.01,description=' ', layout = layStretch)


flag1 = widgets.Checkbox(value=True, description='Use Test Pipeline', layout = layCenter)
flag2 = widgets.Checkbox(value=False, description='Show Computational Timing', layout = layCenter)
flag3 = widgets.Checkbox(value=False, description='Use Morphs', layout = layCenter)

In [328]:
def thresholding(_img, dx=0,dy=3, sz1=5,sz2=10,tu0=0.15,tu1=0.1,tv0=5, tv1=70, tdu = 0.4, tdu2 = 0.1, tdv = 0.15,
    k11=15,k12=2,k21=50,k22=5,k31=75,k32=1, win1=5, win1thr = 0.1, searchWin=10, flag_test=True, flag_timing=False,flag_morph=True
):
    
    vboat.dW = sz2;        vboat.dH = sz1
    vboat.dead_x = dx;     vboat.dead_y = dy
    vboat.dThs = [tdu,tdu2,win1thr,tdv]
    vboat.dHplus = win1

    vboat.kszs[0] = (k11,k12)
    vboat.kszs[1] = (k21,k22)
    vboat.kszs[2] = (k31,k32)
    vboat.testLowCntThresh = k21
    vboat.testHighCntThresh = k31
    vboat.testRestThreshRatio = tdu
    vboat.flag_morphs = flag_morph

    vboat.vmap_search_window_height = searchWin

    vboat.pipelineTest(_img, threshU1=tu0,threshU2=tu1, threshV1=tv0, threshV2=tv1, timing=flag_timing)
    
    display_obstacles = cv2.cvtColor(vboat.img, cv2.COLOR_GRAY2BGR)
    display_obstacles = cv2.applyColorMap(display_obstacles,cv2.COLORMAP_JET)
    [cv2.rectangle(display_obstacles,ob[0],ob[1],(255,255,255),1) for ob in vboat.obstacles]

    overlay = make_uv_overlay(display_obstacles,vboat.umap_deadzoned,vboat.vmap_raw)
    overlay = cv2.cvtColor(overlay,cv2.COLOR_BGR2RGB)
    pplot(overlay,'Overlay')

    dispU1,dispU2,compU = vboat.umap_displays()
    dispV1,dispV2,compV = vboat.vmap_displays()

    dispU2 = cv2.cvtColor(dispU2,cv2.COLOR_BGR2RGB)
    dispV2 = cv2.cvtColor(dispV2,cv2.COLOR_BGR2RGB)
    pplot(dispU2,'Umap Display')
    pplot(dispV2,'Vmap Display')

    distances = []
    angles = []
    umap = vboat.umap_raw
    xs = vboat.xBounds
    ds = np.array(vboat.dbounds)
    obs = vboat.obstacles
    nObs = len(ds)
    if(nObs is not 0):
        for i in range(nObs):
            disparities = ds[i]
            us = [obs[i][0][0], obs[i][1][0]]
            vs = [obs[i][0][1], obs[i][1][1]]
            z,ux,uy,uz = vboat.calculate_distance(umap,us,disparities,vs)

            theta = math.acos((uz/z))

            distances.append(z)
            angles.append(theta)
            print("Obstacle [%d] Distance, Angle: %.3f, %.3f" % (i,z,np.rad2deg(theta)))

In [329]:
%reload_ext autoreload
plt.close('all')

p = interactive(thresholding,
    _img = imgPathSelector, dx = sz1Selector, dy=sz2Selector, sz1=sz3Selector,sz2=sz4Selector,
    tu0 = u1threshSelector,tu1 = u2threshSelector, tv0 = v1threshSelector, tv1 = v2threshSelector,
    tdu = d1threshSelector, tdu2=d2threshSelector, tdv = d3threshSelector, k11=k11Selector,k12=k12Selector,
    k21=k21Selector,k22=k22Selector,k31=k31Selector,k32=k32Selector,win1=win1Selector,win1thr=win1ThreshSelector,
    searchWin=(0,40,1), flag_test = flag1, flag_timing = flag2, flag_morph = flag3
)

rows = []
rows.append(HBox([p.children[0]]))
rows.append(HBox([lblSz1,p.children[1], p.children[2]], layout = layStretch))
rows.append(HBox([lblSz2,p.children[3], p.children[4]], layout = layStretch))
rows.append(HBox([lblU, p.children[5], p.children[6]], layout = layStretch))
rows.append(HBox([lblV, p.children[7], p.children[8]], layout = layStretch))
rows.append(HBox([lblD, p.children[9], p.children[10], p.children[11]], layout = layStretch))
rows.append(HBox([p.children[12], p.children[13]], layout = layStretch))
rows.append(HBox([p.children[14], p.children[15]], layout = layStretch))
rows.append(HBox([p.children[16], p.children[17]], layout = layStretch))
rows.append(HBox([p.children[18], p.children[19]], layout = layStretch))

rows.append(HBox([p.children[-4], p.children[-3], p.children[-2]], layout = layCenter))

controls = VBox(rows, layout = Layout(display='flex'))
output = p.children[-1]

tabs = widgets.Tab([controls, output])
tabs.set_title(0, 'Parameters')
tabs.set_title(1, 'StdOuts')
display(tabs)

VGFiKGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShIQm94KGNoaWxkcmVuPShTZWxlY3QoZGVzY3JpcHRpb249dSdGaWxlOicsIGxheW91dD1MYXlvdXQod2lkdGg9dScxMDAlJyksIG9wdGlvbnPigKY=


In [297]:
vboat.debug_obstacle_search = True
vboat.debug_contours = True

In [325]:
copyImg = np.copy(vboat.img)
testKernel = cv2.getStructuringElement(cv2.MORPH_RECT,(10,10))
testImg = cv2.morphologyEx(copyImg, cv2.MORPH_CLOSE, testKernel)
pplots([copyImg, testImg],"Raw Img Morphing", size=(1,2))

In [321]:
verbose = True;     n = 6
umap = np.copy(vboat.umap_deadzoned);   stripsU = strip_image(umap, nstrips=n);    strip = np.copy(stripsU[0])
# Stripping
hs, ws = strip.shape[:2];              dh = hs / n
# print("strip.shape: ",strip.shape[:2])

dead_strip = strip[0:dh, :];           oDead = np.copy(dead_strip)
rest_strip = strip[dh:hs, :];          oRest = np.copy(rest_strip)
# print("Shapes -- deadStrip, restStrip: %s, %s" % (str(dead_strip.shape),str(rest_strip.shape)))

hd, wd = dead_strip.shape[:2];          hr, wr = rest_strip.shape[:2]
umapMax = np.max(umap); stripMax = np.max(strip); preDeadMax = np.max(dead_strip); restMax = np.max(rest_strip)
print("umapMax, stripMax,preDeadMax, restMax: %d, %d, %d, %d" % (umapMax, stripMax,preDeadMax,restMax))

tmpdead = dead_strip[0:3, :];        tmprest = dead_strip[3:hd, :]
tmaxs = [np.max(tmpdead), np.max(tmprest)]
plist("Pre-Maxs:\t",tmaxs)

ddzoneCopy=np.copy(tmpdead); restCopy = np.copy(tmprest)
tmpdeadD = tmpdead[0:1, :];     tmprestD = tmpdead[1:tmpdead.shape[0], :]
# pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1))
pplots([tmpdead, tmprest],"Before Deadzone Strip Filtering", size=(2,1),flag_resize=True,scale=(5,1))
# pplots([tmpdeadD, tmprestD],"Before Deadzone Deadzone Filtering", size=(2,1),flag_resize=True,scale=(5,1))

mx0 = np.max(ddzoneCopy)
mx1 = np.max(tmpdeadD)
mx2 = np.max(tmprestD)

mn0 = np.mean(ddzoneCopy)
mn1 = np.mean(tmpdeadD)
mn2 = np.mean(tmprestD)
# print(mn0,mn1,mn2,mx0,mx1,mx2)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
tmpRatio2 = mn2/float(mx2)
tmpRatio3 = tmpRatio2 + tmpRatio0*tmpRatio2
# print(tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3)

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(tmpRatio1*mx1)
tmpTH2 = int(tmpRatio2*mx2)
tmpTH3 = int(tmpRatio0*mx0)
tmpTH4 = int(tmpRatio3*mx0)
# print(tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4)

tt1 = ddzoneCopy[0:1, :];     tt2 = ddzoneCopy[1:ddzoneCopy.shape[0], :]

_,testttt1 = cv2.threshold(tt1, tmpTH0,255,cv2.THRESH_TOZERO)
_,testttt2 = cv2.threshold(tt2, tmpTH4,255,cv2.THRESH_TOZERO)

testD = np.concatenate((testttt1,testttt2), axis=0);
tmpdead2 = np.copy(testD)

mx0 = np.max(restCopy);     mn0 = np.mean(restCopy)
mx1 = np.max(testD);        mn1 = np.mean(testD)
gain = mx0/float(mx1)
print("Max Gain:\t%.3f" % gain)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
tmpRatio2 = tmpRatio1+tmpRatio0
tmpRatio3 = tmpRatio0 + gain

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(math.ceil(tmpRatio1*mx1))
tmpTH3 = int(tmpRatio3*tmpTH0)
tmpTH4 = int(tmpRatio3*mx0)
tmpTH5 = tmpTH3 + tmpTH4
print(tmpTH0,tmpTH1,tmpTH2,tmpTH3,tmpTH4,tmpTH5)
combinedThresh = tmpTH5
restThresh = tmpTH1

print(vboat.is_ground_present)
if not vboat.is_ground_present or 100 < combinedThresh < 255: testThresh1 = tmpTH4
elif combinedThresh >= 255: testThresh1 = int(0.5*255)
else: testThresh1 = tmpTH5

print("testThresh1: %d" % testThresh1)

_,tmprest = cv2.threshold(tmprest, testThresh1,255,cv2.THRESH_TOZERO)
# _,tmprest = cv2.threshold(tmprest, tmpTH5,255,cv2.THRESH_TOZERO)

# pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE",size=(2,1),flag_resize=True,scale=(5,1))
pplots([restCopy,tmprest],"Rest of deadzone",size=(2,1),flag_resize=True,scale=(5,1))

ksz2 = (20,1);        ksz1 = (30,1)
kernelr = cv2.getStructuringElement(cv2.MORPH_RECT,ksz1)
kernelD = cv2.getStructuringElement(cv2.MORPH_RECT,ksz2)

tmpdead2 = cv2.morphologyEx(tmpdead2, cv2.MORPH_CLOSE, kernelr)
tmprest = cv2.morphologyEx(tmprest, cv2.MORPH_CLOSE, kernelD)
pplots([tmpdead2,tmprest],"Thresholded Deadzone Strips w/ CLAHE and Morphology",size=(2,1),flag_resize=True,scale=(5,1))

dead_strip = np.concatenate((tmpdead2,tmprest), axis=0);
oStrip = np.concatenate((dead_strip,rest_strip), axis=0);
oDead2 = np.copy(dead_strip)

# pplots([dead_strip],"Tmp",size=(1,1),flag_resize=True,scale=(5,1))
# pplot(dead_strip,"Tmp")

tCLAH2 = cv2.createCLAHE(clipLimit=10.0,tileGridSize=(hs,ws/4))
tstr = tCLAH2.apply(oStrip)

mx0 = np.max(tstr);     mn0 = np.mean(tstr)
mx1 = np.max(oStrip);   mn1 = np.mean(oStrip)
meanGain = (mn0 - mn1)/float(mn1)
print("Means:\t\t%.2f, %.2f"% (mn0,mn1))
print("Maxs:\t\t%d, %d"% (mx0,mx1))
print("Mean Gain:\t%.3f" % meanGain)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
tmpRatio2 = tmpRatio1*meanGain
tmpRatio3 = tmpRatio0 + tmpRatio2
trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]
plist("Tmp Ratios:\t",trats,dplace=3)

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(tmpRatio0*mx0)
tmpTH2 = int(tmpRatio3*mx0)
tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3]
plist("Tmp Threhsolds: ",tThs)

# testThresh = 93
testThresh = tmpTH3

_,tstrMask = cv2.threshold(tstr, testThresh,255,cv2.THRESH_BINARY)
_,ttstr = cv2.threshold(tstr, testThresh,255,cv2.THRESH_TOZERO)


kernelM = cv2.getStructuringElement(cv2.MORPH_RECT,(40,3))
kernelM2 = cv2.getStructuringElement(cv2.MORPH_RECT,(10,2))
ttstrM = cv2.morphologyEx(ttstr, cv2.MORPH_CLOSE, kernelM)
tstrMask = cv2.morphologyEx(tstrMask, cv2.MORPH_CLOSE, kernelM)

tmp = np.copy(oStrip);     stre = cv2.bitwise_and(tmp,tmp,mask=tstrMask)

tmpH = stre.shape[0];      h0 = dead_strip.shape[0]
tmpStrip = np.concatenate((dead_strip,stre[h0:tmpH,:]), axis=0)

print(strip.shape, tmpStrip.shape)
pplots([rest_strip,tstr,ttstr,ttstrM,stre,tmpStrip],"1st Umap Strip Deadzone Filtered",(6,1))

newStrip = np.copy(tmpStrip)

_,newStrip = cv2.threshold(newStrip, restThresh,255,cv2.THRESH_TOZERO)

pStrip = np.copy(newStrip)
# Find Contours
contours,_ = vboat.find_contours(pStrip, 45.0)
disp = cv2.applyColorMap(pStrip,cv2.COLORMAP_PARULA);       disp = cv2.cvtColor(disp,cv2.COLOR_BGR2RGB)
[cv2.drawContours(disp, [cnt], 0, (255,0,0), 1) for cnt in contours]
print("Found %d Objects in 1st Strip" % len(contours))
# pplots([stripD,stripR,copyCld,copyClr],"CLAHE Sections",(4,1))
pplots([strip,newStrip,disp],"Test Compare",(3,1))

umapMax, stripMax,preDeadMax, restMax: 255, 255, 255, 13
Pre-Maxs:	254, 255
Max Gain:	1.004
(241, 8, 29, 254, 269, 523)
True
testThresh1: 127
Means:		35.65, 1.38
Maxs:		255, 255
Mean Gain:	24.801
Tmp Ratios:	0.14, 0.005, 0.134, 0.274
Tmp Threhsolds: 219, 35, 69, 80
((42, 640), (42, 640))
Found 3 Objects in 1st Strip


In [314]:
mx0 = np.max(restCopy);     mn0 = np.mean(restCopy)
mx1 = np.max(testD);        mn1 = np.mean(testD)

mmxs = [mx0,mx1]
mmns = [mn0,mn1]
meanGain = (mn1 - mn0)/float(mn0)
gain = mx0/float(mx1)
plist("Means:\t\t",mmns)
plist("Maxs:\t\t", mmxs)
print("Mean Gain:\t%.3f" % meanGain)
print("Max Gain:\t%.3f" % gain)

tmpRatio0 = mn0/float(mx0)
tmpRatio1 = mn1/float(mx1)
# tmpRatio2 = tmpRatio1*meanGain
# tmpRatio3 = tmpRatio0 + tmpRatio2
tmpRatio2 = tmpRatio1+tmpRatio0
tmpRatio3 = tmpRatio0 + gain
trats = [tmpRatio0,tmpRatio1,tmpRatio2,tmpRatio3]
# trats = [tmpRatio0,tmpRatio1,tmpRatio3]

plist("Tmp Ratios:\t",trats,dplace=4)

tmpTH0 = int((1-tmpRatio0)*mx0)
tmpTH1 = int(math.ceil(tmpRatio1*mx1))
tmpTH2 = int(tmpRatio2*mx0)
tmpTH3 = int(tmpRatio3*tmpTH0)
tmpTH4 = int(tmpRatio3*mx0)
tmpTH5 = tmpTH3 + tmpTH4
# tmpTH3 = int(tmpTH2 + tmpRatio3*tmpTH1 + math.ceil(tmpRatio1*mx0))
tThs = [tmpTH0,tmpTH1,tmpTH2,tmpTH3, tmpTH4,tmpTH5]
plist("Tmp Threhsolds: ",tThs)

# tt1 = ddzoneCopy[0:1, :];     tt2 = ddzoneCopy[1:ddzoneCopy.shape[0], :]

# _,testttt1 = cv2.threshold(tt1, tmpTH0,255,cv2.THRESH_TOZERO)
# _,testttt2 = cv2.threshold(tt2, tmpTH3,255,cv2.THRESH_TOZERO)

# testD = np.concatenate((testttt1,testttt2), axis=0);

# pplots([ddzoneCopy,testD],"Test ddzones", (2,1),flag_resize=True,scale=(5,1))

Means:		13.17, 7.55
Maxs:		255, 254
Mean Gain:	-0.427
Max Gain:	1.004
Tmp Ratios:	0.0517, 0.0297, 0.0814, 1.0556
Tmp Threhsolds: 241, 8, 20, 254, 269, 523


In [307]:
37*(1-0.286)
0.65*255

165.75

In [ ]:
dm = vboat.dmax
pd = vboat.dmax/256.0
pd = 100/256.0
tmpN = pd*6
print(pd)
print(tmpN)
tmp = math.ceil(tmpN)
print(tmp)

In [ ]:
tmpCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(40,ws))
clStrips = [tmpCLAH.apply(strip) for strip in stripsU]

pplots(stripsU,"UStrips", (n,1))
pplots(clStrips,"clStrips", (n,1))

In [ ]:
def test_vstrip_filter():
    vmap = np.copy(vboat.vmap_raw)
    vmax = np.max(vmap)

    testStrips = []
    stripsPv = []
    threshsV = [0, 70, 40,60,60]
    nThreshsV = int(math.ceil((vboat.dmax/256.0) * len(threshsV)))


    stripsV = strip_image(vboat.vmap_deadzoned, nstrips=nThreshsV, horizontal_strips=False)
    pplots(stripsV,"Raw Vmap Strips",(1,nThreshsV))

    # RAW VStrip 1
    testStrips.append(stripsV[0])

    stripMax = np.max(stripsV[0])
    relRatio = stripMax/float(vmax)


    for i, strip in enumerate(stripsV):
        _, tmpStrip = cv2.threshold(strip, threshsV[i], 255, cv2.THRESH_TOZERO)
        stripsPv.append(tmpStrip)

#     pplots(stripsPv,"Pre-Filtered Vmap Strips",(1,nThreshsV))

    vsThs = np.array([
        [0.25, 0.0],
        [0.5, 0.0]
    ])

    # Filtered VStrip 1 STEP 0
    testStrips.append(np.copy(stripsPv[0]))

    tmpMax = np.max(stripsPv[0])

    # Filtered VStrip 1 STEP 1
    testStrips.append(np.copy(stripsPv[0]))

    tmpStrip = stripsPv[0]
    hs, ws = tmpStrip.shape[:2]

    tmpdead = tmpStrip[:, 0:3]
    tmprest = tmpStrip[:, 3:ws]

    deadMax = np.max(tmpdead)
    restMax = np.max(tmprest)
    dmRatio = deadMax / float(stripMax)
    rmRatio = restMax/float(stripMax)
    print("stripMax,vmapMax,deadMax,restMax: %d, %d, %d, %d" % (stripMax,vmax,deadMax,restMax))
    print("relRatio, dmRatio,rmRatio: %.3f, %.3f, %.3f" % (relRatio, dmRatio,rmRatio))
#     pplots([tmpdead,tmprest], "Pre-CLAHE Vmap", (1,2))

    # _, fa = cv2.threshold(tmpdead, tfa,255,cv2.THRESH_TOZERO) #THRESH_TOZERO

    tCLAH = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(5,1))
    tCLAH2 = cv2.createCLAHE(clipLimit=2.0,tileGridSize=(hs,3))
    dCl = tCLAH.apply(tmpdead)
    dCl2 = tCLAH2.apply(tmprest)

#     pplots([dCl,dCl2],"DCLs V",(1,2))
    pc1 = dCl[:, 0:1]
    pc2 = dCl[:, 1:dCl.shape[1]]

    # _,pc1 = cv2.threshold(pc1, 200,255,cv2.THRESH_TOZERO)
    _,pc1 = cv2.threshold(pc1, 150,255,cv2.THRESH_TOZERO)
    _,pc2 = cv2.threshold(pc2, 0,255,cv2.THRESH_TOZERO)
    tmpdead = np.concatenate((pc1,pc2), axis=1)

#     pplots([pc1,pc2],"dCl strips",(1,2))


    cldMax = np.max(tmpdead)
    clrMax = np.max(dCl2)



    # deadThreshGain = (stripMax - deadMax)/255.0
    # deadThreshOffset = int(base_thresh*deadThreshGain)
    # if not vboat.is_ground_present: deadThresh = deadThreshOffset
    # else: deadThresh = base_thresh + deadThreshOffset

    # dead_strip[dead_strip < deadThresh] = 0
    # deadMax = np.max(dead_strip)


    deadMaxGain = (cldMax - deadMax)/float(cldMax)
    restMaxGain = (clrMax - restMax)/float(clrMax)
    print("cldMax,clrMax,deadMaxGain,restMaxGain: %d, %d, %.3f, %.3f" % (cldMax,clrMax,deadMaxGain,restMaxGain))
    newStripMax = int(stripMax*restMaxGain);        newRmRatio = (restMax)/float(newStripMax)
    print("newStripMax, newRmRatio: %d, %.3f" % (newStripMax,newRmRatio))

    ratio_thresh = 0.25;   base = 64
    g1 = restMaxGain*ratio_thresh;         g2 = (g1+ratio_thresh);         g3 = 1 - g1;         g4 = 1 - g2
    t1 = int(g1*clrMax);      t2 = int(g2*clrMax);      t3 = int(g3*clrMax);      t4 = int(g4*clrMax)
    t21 = int(g1*base);       t22 = int(g2*base);       t23 = int(g3*base);       t24 = int(g4*base)
    t31 = int((g2+g2*restMaxGain)*clrMax)
    print; print("g1,g2,g3,g4:\t%.3f, %.3f, %.3f, %.3f" % (g1,g2,g3,g4));
    print("t1,t2,t3,t4:\t%d, %d, %d, %d" % (t1,t2,t3,t4))
    print("t21,t22,t23,t24,t31:\t%d, %d, %d, %d, %d" % (t21,t22,t23,t24,t31));print

    _,tmprest = cv2.threshold(dCl2, t31,255,cv2.THRESH_TOZERO)
    stripsPv[0] = np.concatenate((tmpdead,tmprest), axis=1)


    # Filtered VStrip 1 STEP 2
    testStrips.append(np.copy(stripsPv[0]))

    dw = 5;        vdTh = [0.15, 0]

    # dead_strip = stripsPv[0][:, 0:dw]
    # rest_strip = stripsPv[0][:,dw:stripsPv[0].shape[1]]

    # print("[INFO] Shapes stripsPv[0], dead_strip, rest_strip: %s, %s, %s" % (str(stripsPv[0].shape), str(dead_strip.shape),str(rest_strip.shape)))

    # dead_strip[dead_strip < tmpMax*vdTh[0]] = 0
    # rest_strip[rest_strip < tmpMax*vdTh[1]] = 0
    # stripsPv[0] = np.concatenate((dead_strip,rest_strip), axis=1)

    # # Filtered VStrip 1 STEP 3
    # testStrips.append(np.copy(stripsPv[0]))

    # print("[INFO] stripsPv[0] shape: %s" % (str(stripsPv[0].shape)))

    # stripsPv[1][stripsPv[1] < np.max(stripsPv[1]*0.9)] = 0
    tmpMax = np.max(stripsPv[1])
    tH = stripsPv[1].shape[0]
    top_strip = stripsPv[1][0:tH/3, :]
    bot_strip = stripsPv[1][tH/3:tH, :]

    top_strip[top_strip < tmpMax*vsThs[1,0]] = 0
    bot_strip[bot_strip < tmpMax*vsThs[1,1]] = 0
    stripsPv[1] = np.concatenate((top_strip,bot_strip), axis=0)

    newV = np.concatenate(stripsPv, axis=1)

#     pplots(testStrips,"1st Vmap Strip Filter Steps",(1,len(testStrips)))

    tmp = np.copy(newV)
    vmap = cv2.bitwise_and(tmp,tmp,mask=cv2.bitwise_not(vboat.vmask))
    pplot(vmap,"New Vmap")
    obs, obsU, ybounds, dbounds, windows, nObs = vboat.find_obstacles(vmap, vboat.disparityBounds, vboat.xBounds, ground_detected=vboat.is_ground_present,verbose=True)

    dispV = cv2.applyColorMap(np.copy(vmap),cv2.COLORMAP_JET)
    for wins in windows:
        for win in wins:
            cv2.rectangle(dispV,win[0],win[1],(0,255,255), 1)
    dispV = cv2.cvtColor(dispV,cv2.COLOR_BGR2RGB)
    pplot(dispV,"dispV")


    displayObs = cv2.cvtColor(np.copy(vboat.img), cv2.COLOR_GRAY2BGR)
    displayObs = cv2.applyColorMap(displayObs,cv2.COLORMAP_JET)

    for ob in obs:
        cv2.rectangle(displayObs,ob[0],ob[1],(255,255,255),1)


    overlay = cv2.cvtColor(displayObs,cv2.COLOR_BGR2RGB)
    pplot(overlay,'Test Obs')